# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

2025-10-30 10:05:39,952 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [2]:
in_memory_api_prompt = f"""
Act as a senior Python backend developer. Produce ONE self-contained FastAPI script (main_in_memory.py) implementing an in-memory CRUD API for the users table defined in the SQL schema below.

SQL schema (source context):
{sql_schema}

Requirements:
1. Infer only the users table columns (ignore others). Map SQL types to Python/Pydantic types:
   - INTEGER -> int
   - TEXT/VARCHAR -> str
   - BOOLEAN -> bool
   - DATETIME/TIMESTAMP -> datetime
   - Handle nullable columns as Optional.
2. Pydantic models:
   - UserBase: non-ID, non-timestamp business fields.
   - UserCreate(UserBase): same as base but enforce required (exclude id, created_at, updated_at).
   - UserUpdate: all user-settable fields Optional for partial update.
   - User: full response with id + timestamps.
3. In-memory storage:
   - users_db: list[dict].
   - next_id: incremental int starting at 1.
   - Use threading.RLock for thread safety around mutations.
4. Timestamps:
   - If created_at / updated_at columns exist, populate created_at on create (UTC), update updated_at on any modification.
5. Email uniqueness:
   - Reject create or update that introduces duplicate email with HTTP 400.
6. Endpoints (use response_model, proper status codes):
   - POST /users/ -> 201 (create)
   - GET /users/ -> list all
   - GET /users/user_id -> 404 if missing
   - PUT /users/user_id -> update existing (merge semantics: only provided fields change)
   - DELETE /users/user_id -> 204 no content
7. Implementation details:
   - Internal storage = dicts; convert to Pydantic model for output.
   - Central helper: get_user_or_404(user_id).
   - Raise HTTPException for errors (404, 400).
8. Code quality:
   - Clear functions, minimal duplication.
   - Type hints everywhere.
   - No external dependencies beyond FastAPI, pydantic, datetime, typing.
   - Follow **SOLID** principles and best practices.
9. Main guard:
   if __name__ == "__main__": print startup hint for running: uvicorn main_in_memory:app --reload
10. Output ONLY valid Python code.

Generate the final Python file now.
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/main_in_memory.py", overwrite=True)
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---
# main_in_memory.py

import threading
from datetime import date, datetime, timezone
from enum import Enum
from typing import List, Optional, Dict, Any

from fastapi import FastAPI, HTTPException, Response, status
from pydantic import BaseModel, EmailStr, Field

# -------------------
# Configuration & In-Memory "Database"
# -------------------

# In-memory storage for users, mimicking a database table.
# A list of dictionaries is used for simplicity.
users_db: List[Dict[str, Any]] = []

# A global counter for new user IDs, mimicking AUTOINCREMENT.
next_user_id: int = 1

# A reentrant lock to ensure thread safety for concurrent access to the database.
db_lock = threading.RLock()


# -------------------
# Pydantic Models (Data Transfer Objects)
# -------------------

class UserType(str, Enum):
    """Enumeration for the user_type field, based on the SQL CHECK constraint."""
    NEW_HIRE = 'new_hire'
    MANAGER = 'manager'
    HR_SPEC

## Added test to verify that the endpoints are properly working with generated code at this step

In [3]:
# Test validating correct in memory database /users GET and POST functionality.
import pytest
from fastapi.testclient import TestClient

from artifacts.app.main_in_memory import app  # adjust import if path differs

testClient = TestClient(app)

def test_create_three_users_and_list():
    # Cleanup: delete any existing users via API
    existing = testClient.get("/users/")
    assert existing.status_code == 200
    for u in existing.json():
        resp = testClient.delete(f"/users/{u['id']}")
        assert resp.status_code == 204
        
    payloads = [
        {
            "full_name": "Jane Doe",
            "email": "jane.doe@example.com",
            "user_type": "new_hire",
            "start_date": "2024-08-01",
            "sso_user_id": "auth0|123456",
            "role_id": 10,
            "department_id": 5,
            "manager_id": 1,
            "mentor_id": 2,
            "experience_level": "Mid-level",
            "is_active": True
        },
        {
            "full_name": "John Smith",
            "email": "john.smith@example.com",
            "user_type": "employee",
            "start_date": "2024-09-01",
            "experience_level": "Junior",
            "is_active": True
        },
        {
            "full_name": "Alice Manager",
            "email": "alice.manager@example.com",
            "user_type": "manager",
            "start_date": "2024-07-15",
            "experience_level": "Senior",
            "is_active": True
        }
    ]

    created_ids = []
    for p in payloads:
        r = testClient.post("/users/", json=p)
        assert r.status_code == 201, r.text
        data = r.json()
        created_ids.append(data["id"])
        # basic field checks
        assert data["email"] == p["email"]
        assert data["user_type"] == p["user_type"]
        assert data["full_name"] == p["full_name"]
        assert data["is_active"] == p["is_active"]

    # Verify all users exist
    list_resp = testClient.get("/users/")
    assert list_resp.status_code == 200
    users = list_resp.json()
    assert len(users) == 3
    returned_emails = {u["email"] for u in users}
    expected_emails = {p["email"] for p in payloads}
    assert returned_emails == expected_emails

    # Optional: ensure IDs are unique
    assert len(set(created_ids)) == 3
    print("In-memory API test passed: created and listed 3 users successfully.")

test_create_three_users_and_list()

In-memory API test passed: created and listed 3 users successfully.


### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [4]:
db_code_prompt = f"""
Act as a senior Python backend engineer. Using ONLY the provided SQLite schema context below, generate TWO distinct, well-commented Python code blocks:

Schema context (authoritative, do not infer missing tables):
{sql_schema}


General Requirements (apply to BOTH blocks):
- Target Python 3.11+ and SQLAlchemy 2.x style imports.
- Respect UNIQUE, NOT NULL, CHECK, FOREIGN KEY constraints exactly as in schema.
- Preserve composite PKs and indexes.
- Self-referential foreign keys (manager_id, mentor_id) should have relationship definitions (e.g., manager = relationship("User", remote_side="User.id", ...)).
- Represent is_active (INTEGER 0/1) as Boolean with appropriate server_default.
- Timestamps stored as TEXT remain String columns; do NOT convert to DateTime (keep parity with existing DB).
- Include all required relationships for the user table to function properly.


OUTPUT BLOCK 1 (Models):
- Begin with a comment banner ### SQLAlchemy Models ###
- Provide Base = declarative_base()
- Define ALL tables as Python classes exactly matching columns and constraints.
- Add Index definitions using Index(...) matching those in schema.
- No session or engine code in this block.
- End the block with a comment ### END MODELS ###

OUTPUT BLOCK 2 (Session / Dependency):
- Begin with ### Database Session Setup ###
- Create engine for local SQLite file onboarding.db 
- Create SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False, expire_on_commit=False)
- Provide get_db() dependency generator for FastAPI:
- End with ### END SESSION SETUP ###

Formatting / Output Constraints:
- Output ONLY the two Python code blocks in order.
- Each block must be valid standalone Python.
- NO narrative explanation outside the code blocks.
- DO NOT include FastAPI endpoints or Pydantic models.

Generate now.
"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    save_artifact(generated_db_code, "app/database_models.py", overwrite=True)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models and Session Code ---

--- Generated Database Code ---
### SQLAlchemy Models ###
import typing
from typing import List, Optional

from sqlalchemy import (
    Boolean,
    CheckConstraint,
    ForeignKey,
    Index,
    Integer,
    PrimaryKeyConstraint,
    String,
    Text,
    text,
)
from sqlalchemy.orm import Mapped, declarative_base, mapped_column, relationship

# Base class for declarative models
Base = declarative_base()


class Department(Base):
    """Represents an organizational department."""
    __tablename__ = "departments"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String, unique=True, nullable=False)
    created_at: Mapped[str] = mapped_column(
        Text, nullable=False, server_default=text("strftime('%Y-%m-%dT%H:%M:%fZ', 'now')")
    )
    updated_at: Mapped[str] = mapped_column(
        Text, nullable=False, server_default=text("strftime('%Y-%m-%dT%H:%M:%fZ', 'now')")
    )

    # Rel

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

In [ ]:
# load context artifacts
in_memory_code = load_artifact("app/main_in_memory.py")
db_models_code = load_artifact("app/database_models.py")


integration_prompt = f"""
Generate a working FastAPI main.py file by combining the artifacts below. Follow the exact patterns shown.

ARTIFACTS:
In-Memory App: {in_memory_code}
SQLAlchemy Models: {db_models_code}

INTEGRATION RULES:
1. Copy Pydantic models from in-memory app
2. Copy SQLAlchemy User model and get_db() from database models
3. Replace in-memory API endpoint logic to use database connection and SQLAlchemy patterns.

OUTPUT: Complete, runnable main.py file with proper imports, Pydantic models, SQLAlchemy models, and all 5 endpoints using the exact patterns above.
"""

# Request integration code from the integration model
integration_output = get_completion(integration_prompt, client, model_name, api_provider)

integration_output = clean_llm_output(integration_output, language='python')

save_artifact(integration_output, "app/main.py")
print("Saved integrated app to app/main.py")
print(integration_output)


Saved integrated app to app/main.py
# main.py
import typing
from datetime import date, datetime, timezone
from enum import Enum
from typing import List, Optional, Generator, Any, Dict

from fastapi import FastAPI, HTTPException, Response, status, Depends
from pydantic import BaseModel, EmailStr, Field
from sqlalchemy import (
    create_engine,
    Boolean,
    CheckConstraint,
    ForeignKey,
    Integer,
    String,
    Text,
    text,
)
from sqlalchemy.orm import (
    declarative_base,
    Mapped,
    mapped_column,
    relationship,
    Session,
    sessionmaker,
)

# -------------------
# Pydantic Models (Data Transfer Objects / Schemas)
# Copied from the in-memory app artifact.
# -------------------

class UserType(str, Enum):
    """Enumeration for the user_type field."""
    NEW_HIRE = 'new_hire'
    MANAGER = 'manager'
    HR_SPECIALIST = 'hr_specialist'
    EMPLOYEE = 'employee'


class UserBase(BaseModel):
    """
    Base model for a user, containing common fields.
    """

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.